In [1]:
import pandas as pd
import seaborn as sns
import time
import re
import nltk
import math
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
aa_d = {'a':'Ala',
    'c':'Cys',
    'd':'Asp',
    'e':'Glu',
    'f':'Phe',
    'g':'Gly',
    'h':'His',
    'i':'Ile',
    'k':'Lys',
    'l':'Leu',
    'm':'Met',
    'n':'Asn',
    'p':'Pro',
    'q':'Gln',
    'r':'Arg',
    's':'Ser',
    't':'Thr',
    'v':'Val',
    'w':'Trp',
    'y':'Tyr' }

print(aa_d['g'])

Gly


## AMINO ACID CODE TRANSLATION FROM 1 LETTER TO 3 LETTER WORKS!

In [3]:
def amino2amino(aa):
    aa_d = {'a':'Ala',
    'c':'Cys',
    'd':'Asp',
    'e':'Glu',
    'f':'Phe',
    'g':'Gly',
    'h':'His',
    'i':'Ile',
    'k':'Lys',
    'l':'Leu',
    'm':'Met',
    'n':'Asn',
    'p':'Pro',
    'q':'Gln',
    'r':'Arg',
    's':'Ser',
    't':'Thr',
    'v':'Val',
    'w':'Trp',
    'y':'Tyr' }
    
    if aa not in aa_d.keys():
        return 'X'
    else:
        return aa_d[aa]
    
    

In [4]:
# Can convert to lowercase before using as input to function amino2amino()
amino = 'W'
print(amino2amino(amino.lower()))

Trp


In [5]:
seq1 = 'AWKGGF'
aa_1 = list(seq1)
aa_1

['A', 'W', 'K', 'G', 'G', 'F']

In [6]:
aa_seq1 = []
for aa in aa_1:
    ami = amino2amino(aa.lower())
    aa_seq1.append(ami)

In [7]:
for item in aa_seq1:
    print(item)

Ala
Trp
Lys
Gly
Gly
Phe


In [8]:
seq2 = 'WALALALALW'
aa_2 = list(seq2)
aa_2

['W', 'A', 'L', 'A', 'L', 'A', 'L', 'A', 'L', 'W']

In [9]:
aa_seq2 = []
for aa in aa_2:
    ami = amino2amino(aa.lower())
    aa_seq2.append(ami)

In [10]:
for item in aa_seq2:
    print(item)

Trp
Ala
Leu
Ala
Leu
Ala
Leu
Ala
Leu
Trp


In [11]:
seq1a = ' '.join(aa_seq1)
seq1a

'Ala Trp Lys Gly Gly Phe'

In [12]:
seq2a = ' '.join(aa_seq2)
seq2a

'Trp Ala Leu Ala Leu Ala Leu Ala Leu Trp'

In [13]:
proteins = [seq1a,seq2a]
proteins

['Ala Trp Lys Gly Gly Phe', 'Trp Ala Leu Ala Leu Ala Leu Ala Leu Trp']

In [14]:
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(proteins)
# summarize
print(vectorizer.vocabulary_)
print()
print(vectorizer.idf_)

{'ala': 0, 'trp': 5, 'lys': 3, 'gly': 1, 'phe': 4, 'leu': 2}

[ 1.          1.40546511  1.40546511  1.40546511  1.40546511  1.        ]


In [15]:
vector = CountVectorizer()
vectors = vector.fit_transform(aa_seq1,aa_seq2)

In [16]:
vectors

<6x5 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [17]:
data = vector.fit_transform(aa_seq1,aa_seq2).toarray()
print(data)

[[1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]]


In [18]:
print(vector.get_feature_names())

['ala', 'gly', 'lys', 'phe', 'trp']


## HOLD ON, HOLD ON... NO NEED TO CONVERT 1 LETTER AMINO ACID CODE INTO 3 LETTER AMINO ACID CODE,... SIMPLY SET TfidVectorizer( ) TOKEN PATTERN TO LOOK FOR ONLY 1 CHARACTER WORDS AND YOU ARE GOOD TO GO! IMPORTANT REMOVE ENGLISH STOP WORD ARGUMENT WHEN CREATING  TfidVectorizer( ) TO ALLOW ALANINE (A) TO BE COUNTED AS A WORD!

## Important: Use token_pattern = r'\w{1}' when using TfidVectorizer or CountVectorizer when only looking for single letters.

In [19]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1}',  #vectorize 1-character words only
    ngram_range=(1, 1),
    max_features=30000)

In [20]:
seq1 = 'AWKGGFALW'
seq2 = 'WALALALALWKG'
prot_list=[seq1,seq2]

In [21]:
word_vectorizer.fit(prot_list)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=30000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents='unicode',
                sublinear_tf=True, token_pattern='\\w{1}', tokenizer=None,
                use_idf=True, vocabulary=None)

In [22]:
feature_names = word_vectorizer.get_feature_names()
feature_names

['a', 'f', 'g', 'k', 'l', 'w']

In [23]:
vectorizer = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(1, 1))
#vectorizer = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(2, 2))
vectorizer.fit(prot_list)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='\\w{1}', tokenizer=None,
                vocabulary=None)

In [24]:
# See this token pattern,'\\w{1}' , works with single letters which is needed to study amino acid composition!
print("Amino Acids for each protein:")
print(vectorizer.get_feature_names())


Amino Acids for each protein:
['a', 'f', 'g', 'k', 'l', 'w']


In [25]:
x = vectorizer.transform(prot_list)
x = x.toarray()
print("")
print("Transformed Single Amino Acid vector is \n{}".format(x))


Transformed Single Amino Acid vector is 
[[2 1 2 1 1 2]
 [4 0 1 1 4 2]]


In [26]:
vect_2 = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(2,2))
vect_2.fit(prot_list)
print("Dipeptides for each protein:")
print(vect_2.get_feature_names())

Dipeptides for each protein:
['a l', 'a w', 'f a', 'g f', 'g g', 'k g', 'l a', 'l w', 'w a', 'w k']


In [27]:
x2 = vect_2.transform(prot_list)
x2 = x2.toarray()
print("")
print("Transformed Dipeptide vector is \n{}".format(x2))


Transformed Dipeptide vector is 
[[1 1 1 1 1 1 0 1 0 1]
 [4 0 0 0 0 1 3 1 1 1]]


In [28]:
vect_3 = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(3, 3))
vect_3.fit(prot_list)
print("Tripeptides for each protein:")
print(vect_3.get_feature_names())


Tripeptides for each protein:
['a l a', 'a l w', 'a w k', 'f a l', 'g f a', 'g g f', 'k g g', 'l a l', 'l w k', 'w a l', 'w k g']


In [29]:
x3 = vect_3.transform(prot_list)
x3 = x3.toarray()
print("")
print("Transformed tripeptide vector is \n{}".format(x3))


Transformed tripeptide vector is 
[[0 1 1 1 1 1 1 0 0 0 1]
 [3 1 0 0 0 0 0 3 1 1 1]]
